In [1]:
import sys
sys.path.insert(1, '../utils/')
import county_matcher as cm

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

#### Import County Information:

In [3]:
# Still need to format this dataframe - shown below is just a proof-of-concept for the matching algorithm
#Example ONLY -- for actual matching, need to ensure that # of cols in df matches w/ # of answers from user
county_df = pd.read_csv("../data/interim/Terrain_imputed.csv.gz") # test using only terrain data
# county_df = county_df.iloc[:,:17]
feature_cols = county_df.columns[3:]

#### Import User "Responses" and "Importances":

In [4]:
user_df = pd.read_csv("../data/external/user_responses/user_input_2.csv")

#### Find Closest Matching Counties:

In [5]:
n_neighbors=500 # number of neighbors to match
matched_counties = cm.county_matcher(county_df,user_df,feature_cols,target_col="FIPS",n_neighbors = n_neighbors)

c:\ProgramData\Miniconda3\envs\siads699\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [6]:
matched_counties

,FIPS,County,State
3737,6071,San Bernardino County,California
7706,6071,San Bernardino County,California
1316,6071,San Bernardino County,California
7663,4005,Coconino County,Arizona
1273,4005,Coconino County,Arizona
...,...,...,...
4993,27061,Itasca County,Minnesota
7070,27061,Itasca County,Minnesota
2508,27061,Itasca County,Minnesota
4139,36089,St. Lawrence County,New York


In [ ]:
## Next steps / to-do:

# - format county DataFrame to include all the data across all the different datasets
# - format user questions/responses to match the ^^ formatted county data
# - decide how to handle the "year" variable (produces multiple results with same county based on multiple years)